In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
torch.set_default_dtype(torch.float64)
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [4]:
from os import path
from glob import glob
import numpy as np
from pao_tfn_trainer import train_pao_tfn
from se3cnn.point_utils import difference_matrix
from pao_file_utils import parse_pao_file, write_pao_file

In [5]:
# assuming MOLOPT-DZVP as primary basis set
prim_basis_shells = {
    'H': [2, 1, 0], # two s-shells, one p-shell, no d-shells
    'O': [2, 2, 1], # two s-shells, two p-shells, one d-shell
}
pao_basis_size = 4

pao_files = sorted(glob("2H2O_MD/frame_*/2H2O_pao44-1_0.pao"))
training_pao_files = pao_files[:10]

net_H = train_pao_tfn(training_pao_files, prim_basis_shells, pao_basis_size, kind_name="H", max_epochs=500)
net_O = train_pao_tfn(training_pao_files, prim_basis_shells, pao_basis_size, kind_name="O", max_epochs=500)

Training NN for kind H using 40 samples.
Epoch: 0  Missmatch: 65.988971  Penalty: 15.393556 Loss: 81.382527
Epoch: 10  Missmatch: 16.143351  Penalty: 12.687095 Loss: 28.830446
Epoch: 20  Missmatch: 5.557109  Penalty: 11.008619 Loss: 16.565729
Epoch: 30  Missmatch: 4.060342  Penalty: 8.684634 Loss: 12.744976
Epoch: 40  Missmatch: 4.619042  Penalty: 5.986779 Loss: 10.605821
Epoch: 50  Missmatch: 3.431689  Penalty: 2.990734 Loss: 6.422423
Epoch: 60  Missmatch: 3.181948  Penalty: 0.791639 Loss: 3.973587
Epoch: 70  Missmatch: 3.767724  Penalty: 0.733355 Loss: 4.501079
Epoch: 80  Missmatch: 3.047345  Penalty: 0.660740 Loss: 3.708084
Epoch: 90  Missmatch: 2.735382  Penalty: 0.688411 Loss: 3.423793
Epoch: 100  Missmatch: 2.755638  Penalty: 0.341207 Loss: 3.096845
Epoch: 110  Missmatch: 2.486921  Penalty: 0.506027 Loss: 2.992948
Epoch: 120  Missmatch: 3.084738  Penalty: 0.402581 Loss: 3.487320
Epoch: 130  Missmatch: 2.173031  Penalty: 0.694323 Loss: 2.867355
Epoch: 140  Missmatch: 2.504155  Pen

In [6]:
for fn in pao_files:
    kinds, atom2kind, coords, xblocks = parse_pao_file(fn)
    kind_onehot = train_dataset.encode_kind(atom2kind)  # TODO: store kind-encoding in the net
    natoms = coords.shape[0]
    xblocks = []
    for iatom in range(natoms):
        rolled_kinds = np.roll(kind_onehot, shift=-iatom, axis=1)
        rolled_coords =  np.roll(coords, shift=-iatom, axis=0)
        rolled_kinds_torch = torch.from_numpy(rolled_kinds[None,...])
        rolled_coords_torch = torch.from_numpy(rolled_coords[None,...])
        diff_M = difference_matrix(rolled_coords_torch)
        if atom2kind[iatom] == "H":
            output_net = net_H(rolled_kinds_torch, diff_M)
            xblock_net = net_H.decode_xblock(output_net[0,:,0])
        elif atom2kind[iatom] == "O":
            output_net = net_O(rolled_kinds_torch, diff_M)
            xblock_net = net_O.decode_xblock(output_net[0,:,0])
        xblocks.append(xblock_net.detach().numpy())

    frame_dir = path.dirname(fn)
    fn_inferred = frame_dir + "/inferred.pao"
    write_pao_file(coords, xblocks, fn_inferred)
    print("Wrote "+fn_inferred)
    ! cd $frame_dir; /opt/cp2k/exe/local/cp2k.sopt -i 2H2O_pao44_inferred.inp > 2H2O_pao44_inferred.out

NameError: name 'train_dataset' is not defined